[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jeremylongshore/claude-code-plugins-plus-skills/blob/main/tutorials/skills/04-advanced-patterns.ipynb)

# Advanced Skill Patterns

**Learning Path**: Skills → Plugins → Orchestration  
**Level**: Advanced  
**Time**: 45 minutes  
**Prerequisites**: [01-what-is-skill](01-what-is-skill.ipynb), [02-skill-anatomy](02-skill-anatomy.ipynb), [03-build-your-first-skill](03-build-your-first-skill.ipynb)

---

## What You'll Learn

1. ✅ **Multi-phase skills** - Progressive context loading
2. ✅ **Complex tool authorization** - Scoped Bash, restricted operations
3. ✅ **Skill composition** - Skills calling other skills
4. ✅ **Orchestrator patterns** - Managing sub-agents
5. ✅ **Performance optimization** - Context budget management
6. ✅ **Real-world example** - Build a test harness skill

---

## Why Advanced Patterns Matter

**Basic skills** handle single-phase operations:
```yaml
name: simple-formatter
description: Format code files
allowed-tools: Read, Write, Edit
```

**Advanced skills** handle complex workflows:
```yaml
name: test-orchestrator
description: |
  Run full test suite with intelligent retry logic.
  Phase 1: Analyze test structure
  Phase 2: Execute tests in parallel
  Phase 3: Collect results and generate report
allowed-tools: Read, Grep, Bash(test:*), Task
```

**The difference**: Advanced patterns scale to production complexity.

In [ ]:
# Pattern comparison
patterns = {
    "Basic": {
        "phases": 1,
        "tools": "Read, Write",
        "context": "~500 tokens",
        "use_case": "Single file operations"
    },
    "Advanced": {
        "phases": 3,
        "tools": "Read, Grep, Bash(scoped), Task",
        "context": "~2000 tokens (progressive)",
        "use_case": "Multi-step workflows with sub-agents"
    }
}

print("BASIC vs ADVANCED PATTERNS")
print("=" * 60)
for pattern_type, props in patterns.items():
    print(f"\n{pattern_type.upper()}:")
    for key, value in props.items():
        print(f"  {key}: {value}")

print("\n💡 Advanced patterns = Complex workflows made simple!")

## Pattern 1: Multi-Phase Skills

### Problem: Context Budget Limits

Large skills waste context by loading everything upfront:
```markdown
# Bad: All 5000 words load immediately
## Phase 1: Analysis (1000 words)
## Phase 2: Execution (2000 words)  
## Phase 3: Reporting (2000 words)
```

### Solution: Progressive Context Loading

Load only what's needed:
```markdown
# Phase 1: Analysis
[Current phase instructions - 500 words]

**Next**: When analysis complete, user says "continue" to load Phase 2.
```

**Benefits**:
- 80% context reduction
- Faster invocation
- Clearer instructions (no cognitive overload)

In [ ]:
# Multi-phase skill template
def create_multiphase_skill(name, phases):
    """
    Create a multi-phase skill with progressive loading.
    
    Args:
        name: Skill name (kebab-case)
        phases: List of dicts with 'name' and 'instructions'
    """
    # Frontmatter
    frontmatter = f"""---
name: {name}
description: |
  Multi-phase workflow with progressive context loading.
  {', '.join([p['name'] for p in phases])}
allowed-tools: Read, Write, Edit, Grep, Bash(test:*)
version: 1.0.0
license: MIT
author: Your Name <your@email.com>
tags: [advanced, multi-phase, orchestration]
---
"""
    
    # Phase 1 instructions
    body = f"""# {phases[0]['name']}

{phases[0]['instructions']}

**Phase Completion**: When done, inform the user:
```
✅ {phases[0]['name']} complete.
📋 Ready for Phase 2: {phases[1]['name'] if len(phases) > 1 else 'N/A'}
💬 Say 'continue' to proceed.
```

---

*Note: Remaining phases ({len(phases)-1}) load on demand.*
"""
    
    return frontmatter + "\n" + body

# Example: Test harness skill
phases = [
    {
        "name": "Phase 1: Test Discovery",
        "instructions": """Find all test files using Grep.
Analyze test structure and dependencies.
Create execution plan based on priority."""
    },
    {
        "name": "Phase 2: Test Execution",
        "instructions": "Execute tests using planned strategy."
    },
    {
        "name": "Phase 3: Results Analysis",
        "instructions": "Collect results, generate report."
    }
]

skill = create_multiphase_skill("test-orchestrator", phases)
print("MULTI-PHASE SKILL TEMPLATE")
print("=" * 60)
print(skill[:500] + "...\n")
print(f"✅ Only Phase 1 loads initially! (~200 tokens vs ~2000)")

## Pattern 2: Complex Tool Authorization

### Enterprise Security Model

The **6767-c standard** requires scoped tool access:

| Tool Pattern | Security Level | Use Case |
|--------------|----------------|----------|
| `Read, Write` | Safe | File I/O only |
| `Bash(git:*)` | Scoped | Git operations only |
| `Bash(test:*)` | Scoped | Test commands only |
| `Bash` (unscoped) | ❌ BLOCKED | Never allowed |
| `Task` | Requires justification | Sub-agent spawning |

### Scoping Patterns

```yaml
# Good: Specific operations
allowed-tools: Bash(git:status), Bash(git:diff), Bash(git:log)

# Better: Namespace wildcards
allowed-tools: Bash(git:*)

# Best: Multiple namespaces
allowed-tools: Read, Write, Bash(git:*), Bash(test:*)
```

In [ ]:
import re

def validate_tool_authorization(allowed_tools_str):
    """
    Validate tool authorization against 6767-c standard.
    
    Rules:
    1. Must be CSV format (not array)
    2. Bash MUST be scoped (no unscoped Bash)
    3. Task requires explicit justification
    """
    errors = []
    warnings = []
    
    # Parse tools
    tools = [t.strip() for t in allowed_tools_str.split(',')]
    
    for tool in tools:
        # Check for unscoped Bash
        if tool == "Bash":
            errors.append("CRITICAL: Unscoped 'Bash' not allowed. Use Bash(namespace:*) pattern.")
        
        # Check Bash scoping format
        if tool.startswith("Bash("):
            if not re.match(r'^Bash\([a-z-]+:\*?[a-z-]*\)$', tool):
                errors.append(f"Invalid Bash scope: {tool}. Use format: Bash(namespace:*)")
        
        # Check Task usage
        if tool == "Task":
            warnings.append("Task tool detected - ensure skill body justifies sub-agent spawning.")
    
    return errors, warnings

# Test cases
test_cases = [
    ("Read, Write, Edit", "✅ Safe file operations"),
    ("Read, Bash(git:*)", "✅ Scoped git operations"),
    ("Read, Bash", "❌ Unscoped Bash blocked"),
    ("Read, Bash(test:*), Task", "⚠️ Task requires justification"),
]

print("TOOL AUTHORIZATION VALIDATION")
print("=" * 60)
for tools_str, expected in test_cases:
    errors, warnings = validate_tool_authorization(tools_str)
    print(f"\nTools: {tools_str}")
    print(f"Expected: {expected}")
    if errors:
        print(f"Errors: {errors}")
    if warnings:
        print(f"Warnings: {warnings}")
    if not errors and not warnings:
        print("✅ Valid!")

## Pattern 3: Skill Composition

### Skills Calling Other Skills

Advanced skills can **delegate** to specialized skills:

```markdown
# orchestrator-skill

## Step 1: Validate Input
Use the `schema-validator` skill to check input format.

## Step 2: Process Data  
Use the `data-transformer` skill to normalize data.

## Step 3: Generate Output
Use the `report-generator` skill to create final report.
```

**Benefits**:
- Reusable components
- Clear separation of concerns
- Easier testing (test skills independently)
- Progressive context (only load needed skills)

In [ ]:
# Orchestrator skill pattern
orchestrator_skill = """---
name: data-pipeline-orchestrator
description: |
  Orchestrate multi-step data pipeline.
  Delegates to specialized skills for each phase.
allowed-tools: Read, Write, Bash(pipeline:*)
version: 1.0.0
license: MIT
author: Pipeline Team <pipeline@example.com>
tags: [orchestration, composition, pipeline]
---

# Data Pipeline Orchestrator

Execute data pipeline in three phases using specialized skills.

## Phase 1: Validation

**Delegate to**: `schema-validator` skill

1. Read input files using Read tool
2. Invoke schema-validator skill
3. Check validation results
4. If validation fails, stop and report errors

## Phase 2: Transformation

**Delegate to**: `data-transformer` skill

1. Pass validated data to transformer
2. Apply business rules
3. Normalize output format

## Phase 3: Reporting

**Delegate to**: `report-generator` skill

1. Collect transformation results
2. Generate summary statistics
3. Create final report

## Error Handling

At each phase:
- Log errors to pipeline.log
- Stop on critical failures
- Report which skill failed and why
"""

print("SKILL COMPOSITION PATTERN")
print("=" * 60)
print(orchestrator_skill)
print("\n💡 Main skill delegates to 3 specialized skills!")

## Pattern 4: Orchestrator with Sub-Agents

### The Task Tool Pattern

Use the **Task tool** to spawn sub-agents for parallel work:

```markdown
## Parallel Test Execution

1. Discover test files (main agent)
2. Spawn 3 sub-agents using Task tool:
   - Agent A: Run unit tests
   - Agent B: Run integration tests
   - Agent C: Run e2e tests
3. Collect results from all agents
4. Generate unified report (main agent)
```

### When to Use Sub-Agents

| Scenario | Use Sub-Agents? | Why |
|----------|----------------|-----|
| Sequential steps | ❌ No | Main agent is faster |
| Parallel operations | ✅ Yes | Sub-agents run concurrently |
| Independent tasks | ✅ Yes | Isolation prevents conflicts |
| Complex analysis | ✅ Yes | Distribute cognitive load |

### Authorization Requirements

To use Task tool:
```yaml
allowed-tools: Read, Write, Task
```

**Note**: Task tool requires explicit justification in skill body.

In [ ]:
# Sub-agent orchestrator pattern
subagent_orchestrator = """---
name: parallel-test-runner
description: |
  Run test suites in parallel using sub-agents.
  Spawns 3 agents for unit/integration/e2e tests.
allowed-tools: Read, Grep, Bash(test:*), Task
version: 1.0.0
license: MIT
author: Testing Team <test@example.com>
tags: [testing, orchestration, parallel, sub-agents]
---

# Parallel Test Runner

Execute test suites in parallel for faster CI/CD.

## Why Sub-Agents?

**Justification for Task tool**:
- 3 test suites run independently (no shared state)
- Parallel execution reduces runtime from 15min → 5min
- Each agent gets isolated context (prevents interference)

## Phase 1: Test Discovery (Main Agent)

1. Use Grep to find all test files:
   - `tests/unit/**/*.test.ts`
   - `tests/integration/**/*.test.ts`
   - `tests/e2e/**/*.spec.ts`

2. Analyze test dependencies
3. Create execution plan

## Phase 2: Parallel Execution (Sub-Agents)

Spawn 3 sub-agents using Task tool:

### Agent A: Unit Tests
```bash
Task("Run unit tests", "Execute all unit tests in tests/unit/")
```

### Agent B: Integration Tests  
```bash
Task("Run integration tests", "Execute integration tests in tests/integration/")
```

### Agent C: E2E Tests
```bash
Task("Run e2e tests", "Execute e2e tests in tests/e2e/")
```

## Phase 3: Results Collection (Main Agent)

1. Wait for all sub-agents to complete
2. Collect results from each agent
3. Merge coverage reports
4. Generate unified test summary

## Output Format

```
Test Results Summary
====================
Unit Tests:        ✅ 245 passed, 0 failed
Integration Tests: ✅ 89 passed, 0 failed  
E2E Tests:         ✅ 34 passed, 0 failed

Total Runtime: 5m 23s (3x speedup via parallelization)
Coverage: 94.7%
```
"""

print("SUB-AGENT ORCHESTRATOR PATTERN")
print("=" * 60)
print(subagent_orchestrator)
print("\n💡 3 sub-agents run in parallel → 3x faster!")

## Pattern 5: Performance Optimization

### Context Budget Management

**Enterprise limits** (6767-c standard):
- Body: ≤500 lines OR ≤5000 words OR ≤7500 tokens
- Frontmatter: ~200 tokens
- Total skill: <8000 tokens

### Optimization Strategies

| Strategy | Savings | Tradeoff |
|----------|---------|----------|
| Multi-phase loading | 70-80% | Requires user interaction |
| External references | 50-60% | Extra Read calls |
| Skill composition | 40-50% | More skills to maintain |
| Concise instructions | 20-30% | Must stay clear |

### Best Practices

1. **Use examples sparingly** - Show patterns, not exhaustive lists
2. **Link to docs** - Don't duplicate existing documentation
3. **Progressive disclosure** - Load details on demand
4. **Prioritize clarity** - Concise ≠ cryptic

In [ ]:
def analyze_context_usage(skill_content):
    """
    Analyze context usage and provide optimization suggestions.
    """
    # Parse frontmatter and body
    parts = skill_content.split('---')
    if len(parts) < 3:
        return {"error": "Invalid skill format"}
    
    frontmatter = parts[1].strip()
    body = parts[2].strip()
    
    # Count metrics
    body_lines = len(body.split('\n'))
    body_words = len(body.split())
    body_chars = len(body)
    
    # Estimate tokens (rough: 1 token ≈ 4 chars)
    estimated_tokens = body_chars // 4
    frontmatter_tokens = len(frontmatter) // 4
    total_tokens = estimated_tokens + frontmatter_tokens
    
    # Check limits
    limits = {
        "lines": 500,
        "words": 5000,
        "tokens": 7500
    }
    
    violations = []
    if body_lines > limits["lines"]:
        violations.append(f"Lines: {body_lines}/{limits['lines']} (over by {body_lines - limits['lines']})")
    if body_words > limits["words"]:
        violations.append(f"Words: {body_words}/{limits['words']} (over by {body_words - limits['words']})")
    if estimated_tokens > limits["tokens"]:
        violations.append(f"Tokens: {estimated_tokens}/{limits['tokens']} (over by {estimated_tokens - limits['tokens']})")
    
    # Suggest optimizations
    suggestions = []
    if body_lines > 300:
        suggestions.append("Consider multi-phase pattern (70% reduction)")
    if body_words > 3000:
        suggestions.append("Move examples to external docs (50% reduction)")
    if estimated_tokens > 5000:
        suggestions.append("Use skill composition to split into smaller skills (40% reduction)")
    
    return {
        "metrics": {
            "body_lines": body_lines,
            "body_words": body_words,
            "estimated_tokens": estimated_tokens,
            "total_tokens": total_tokens
        },
        "limits": limits,
        "violations": violations,
        "suggestions": suggestions,
        "status": "✅ Within limits" if not violations else "⚠️ Exceeds limits"
    }

# Test with sample skill
sample_skill = """---
name: test-skill
description: Test skill for analysis
allowed-tools: Read, Write
version: 1.0.0
license: MIT
author: Test <test@example.com>
tags: [test]
---

# Test Skill Body

""" + ("Lorem ipsum dolor sit amet. " * 500)  # Simulate large body

analysis = analyze_context_usage(sample_skill)

print("CONTEXT USAGE ANALYSIS")
print("=" * 60)
print(f"Status: {analysis['status']}")
print(f"\nMetrics:")
for key, value in analysis['metrics'].items():
    print(f"  {key}: {value}")

if analysis['violations']:
    print(f"\nViolations:")
    for v in analysis['violations']:
        print(f"  ❌ {v}")

if analysis['suggestions']:
    print(f"\nOptimization Suggestions:")
    for s in analysis['suggestions']:
        print(f"  💡 {s}")

## 🎯 Real-World Example: Build a Test Harness Skill

Let's build a production-ready test harness skill using all advanced patterns:

### Requirements
- Multi-phase: Discovery → Execution → Reporting
- Scoped tools: `Bash(test:*)` for test commands
- Sub-agents: Parallel test execution
- Context optimized: <7500 tokens
- Enterprise compliant: 6767-c standard

In [ ]:
# Production test harness skill
test_harness_skill = """---
name: production-test-harness
description: |
  Enterprise test harness with parallel execution.
  Phase 1: Discover tests and analyze structure
  Phase 2: Execute tests in parallel via sub-agents
  Phase 3: Collect results and generate report
  Use when: Running test suites, CI/CD validation, pre-release checks
allowed-tools: Read, Grep, Bash(test:*), Bash(coverage:*), Task
version: 1.0.0
license: MIT
author: QA Team <qa@example.com>
tags: [testing, orchestration, parallel, enterprise]
model: sonnet
---

# Production Test Harness

Enterprise-grade test execution with parallel sub-agents.

## Phase 1: Test Discovery

**Objective**: Analyze test structure and create execution plan.

### Steps

1. **Find test files**:
   ```bash
   Grep(pattern="(test|spec)\\.(ts|js|py)$", type="test")
   ```

2. **Categorize tests**:
   - Unit: `tests/unit/**/*`
   - Integration: `tests/integration/**/*`  
   - E2E: `tests/e2e/**/*`

3. **Analyze dependencies**:
   - Check for test fixtures
   - Identify setup/teardown requirements
   - Determine execution order

4. **Create execution plan**:
   - Estimate runtime per category
   - Allocate sub-agents (1 per category)
   - Define success criteria

**Output**: Execution plan summary

**Next**: User says "execute" to proceed to Phase 2.

---

## Phase 2: Parallel Execution

**Objective**: Run tests in parallel using sub-agents.

### Sub-Agent Justification

**Why Task tool is needed**:
- 3 test categories run independently (no conflicts)
- Parallel execution: 15min → 5min (3x speedup)
- Isolated contexts prevent interference
- Each agent reports separately for traceability

### Sub-Agent Assignments

**Agent A: Unit Tests**
```
Task(
  description="Run unit tests",
  prompt="Execute all unit tests in tests/unit/. Use Bash(test:npm test unit). Report pass/fail counts.",
  subagent_type="test-automator"
)
```

**Agent B: Integration Tests**
```
Task(
  description="Run integration tests",
  prompt="Execute integration tests in tests/integration/. Use Bash(test:npm test integration). Report results.",
  subagent_type="test-automator"
)
```

**Agent C: E2E Tests**
```
Task(
  description="Run e2e tests",
  prompt="Execute e2e tests in tests/e2e/. Use Bash(test:npm test e2e). Report results.",
  subagent_type="test-automator"
)
```

**Monitoring**: Track agent progress, collect results as they complete.

**Next**: When all agents finish, proceed to Phase 3.

---

## Phase 3: Results & Reporting

**Objective**: Aggregate results and generate comprehensive report.

### Steps

1. **Collect results** from all sub-agents
2. **Merge coverage reports** using Bash(coverage:merge)
3. **Calculate metrics**:
   - Total tests: passed/failed/skipped
   - Code coverage percentage
   - Runtime per category
   - Speedup from parallelization

4. **Generate report**:
   ```
   Production Test Results
   =======================
   
   Unit Tests:        ✅ 245/245 passed (0 failed)
   Integration Tests: ✅ 89/89 passed (0 failed)
   E2E Tests:         ⚠️ 33/34 passed (1 failed)
   
   Overall: 367/368 tests passed (99.7%)
   Coverage: 94.7% (+2.3% from last run)
   Runtime: 5m 23s (3x speedup via parallelization)
   
   Failed Tests:
   - tests/e2e/checkout.spec.ts:42 (timeout)
   ```

5. **Save artifacts**:
   - test-results.json
   - coverage-report.html
   - execution-log.txt

**Output**: Final test report and artifacts

---

## Error Handling

- **Agent failure**: Log which agent failed, retry once
- **Test timeout**: Kill after 10min, report partial results
- **Coverage merge error**: Warn but continue with individual reports

## Success Criteria

- All tests discovered and categorized
- Sub-agents complete successfully
- Coverage ≥90%
- No critical test failures
"""

# Validate the skill
analysis = analyze_context_usage(test_harness_skill)

print("PRODUCTION TEST HARNESS SKILL")
print("=" * 60)
print(test_harness_skill[:800] + "...\n")
print(f"Status: {analysis['status']}")
print(f"Estimated tokens: {analysis['metrics']['estimated_tokens']}/7500")
print(f"\n✅ Ready for production use!")

## 💪 Your Turn: Build an Advanced Skill

Choose one pattern and build a skill:

1. **Multi-phase CI/CD Pipeline**
   - Phase 1: Lint & typecheck
   - Phase 2: Run tests
   - Phase 3: Build & deploy

2. **Code Review Orchestrator**
   - Spawn sub-agents: security scanner, style checker, complexity analyzer
   - Aggregate results into unified review

3. **Documentation Generator**
   - Phase 1: Analyze codebase structure
   - Phase 2: Generate API docs
   - Phase 3: Create user guides

### Success Criteria
- ✅ Follows 6767-c standard
- ✅ Uses appropriate advanced pattern
- ✅ Within context budget (<7500 tokens)
- ✅ Clear justification for tool usage
- ✅ Production-ready instructions

In [ ]:
# Your skill here!
# Use the templates from this notebook as starting points

my_advanced_skill = """---
name: my-advanced-skill
description: |
  [Your description here]
  [Include use cases and trigger phrases]
allowed-tools: [Your tools here]
version: 1.0.0
license: MIT
author: Your Name <your@email.com>
tags: [your, tags]
---

# [Your Skill Name]

[Your instructions here]
"""

# Validate your skill
validation = analyze_context_usage(my_advanced_skill)
print(f"Status: {validation['status']}")
print(f"Tokens: {validation['metrics']['estimated_tokens']}/7500")

# When ready, save to file:
# with open('.claude/skills/my-advanced-skill/SKILL.md', 'w') as f:
#     f.write(my_advanced_skill)

## Key Takeaways

### What You Learned

1. ✅ **Multi-phase skills** - Progressive context loading (70-80% savings)
2. ✅ **Tool authorization** - Scoped Bash, security patterns
3. ✅ **Skill composition** - Delegate to specialized skills
4. ✅ **Sub-agent orchestration** - Parallel execution with Task tool
5. ✅ **Performance optimization** - Context budget management
6. ✅ **Production patterns** - Real-world test harness example

### When to Use Advanced Patterns

- ✅ Multi-step workflows (>3 phases)
- ✅ Parallel operations possible
- ✅ Large context requirements (>2000 tokens)
- ✅ Complex tool authorization needs
- ✅ Reusable component patterns

### Enterprise Standards Compliance

- [x] All required fields present
- [x] Bash is scoped (never unscoped)
- [x] Task tool usage justified
- [x] Context budget <7500 tokens
- [x] Clear phase transitions
- [x] Error handling documented

---

## Next Steps

1. **[05-skill-validation.ipynb](05-skill-validation.ipynb)** - Automated validation
2. **[06-plugin-introduction.ipynb](../plugins/01-what-is-plugin.ipynb)** - Move to plugins
3. **Practice**: Build 3 skills using different patterns

---

*Enterprise Standards Compliant • 6767-c • Version 1.0.0 • MIT License*